In [67]:
import os
import pandas as pd
import numpy as np
from stackregression import stack_regression_step1, stack_regression_step2, print_prediction_report
from utils import encode_numeric_zscore_list, encode_numeric_zscore_all, to_xy
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from scipy.sparse import csr_matrix
from xgboost import XGBRegressor
from random import randint
import xgb

ImportError: No module named 'xgb'

In [65]:
#Read Input CSV file
path = "./data/self"
inputFilePath = os.path.join(path, "TestRegression.csv")
#df = pd.read_csv(inputFilePath, compression="zip", header=0, na_values=['NULL'])
df = pd.read_csv(inputFilePath, header=0, na_values=['NULL'])
x,y = to_xy(df, "Label")
x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.20, random_state=43)


float64


In [66]:
x_train = csr_matrix(x_train)
y_train = csr_matrix(y_train)
x_valid = csr_matrix(x_valid)
y_valid = csr_matrix(y_valid)

xgtrain = xgb.DMatrix(x_train, label=y_train)
xgvalid = xgb.DMatrix(x_valid, label=y_valid)

NameError: name 'xgb' is not defined

In [64]:
#best params on 11/1 for 85% train data: {'subsample': 1.0, 'n_estimators': 174.0, 'eta': 0.1, 
#'colsample_bytree': 0.4, 'gamma': 0.2, 'min_child_weight': 1.0, 'max_depth': 3}

RANDOM_STATE = randint(1,429496)
params = {
    'min_child_weight': 1,
    'eta': 0.01,
    'colsample_bytree': 0.5,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 1,
    'gamma': 1,
    'silent': 1,
    'verbose_eval': False,
    'seed': RANDOM_STATE,
    'eval_metric': 'mae',
    'objective': 'reg:linear',
}

model = XGBRegressor(n_estimators=10000)
model.fit(xgtrain, y_train, eval_metric="mae", early_stopping_rounds=50)

NameError: name 'xgtrain' is not defined

In [3]:
#shuffle dataset. Unnecessary in this case because already sorted by guid
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))

#df.drop('id', axis=1, inplace=True)
#encode categoricals as dummies

#encode_text_dummy_list(df, ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28', 'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35', 'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42', 'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49', 'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56', 'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63', 'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70', 'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77', 'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84', 'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113', 'cat114', 'cat115', 'cat116'])

#encode all numeric values to zscored values
#encode_numeric_zscore(df, 'AW')

#encode_numeric_log_list(df, ['A', 'B', 'C', 'D'])
encode_numeric_zscore_list(df, ['A', 'B', 'C', 'D'])

#discard rows where z-score > 2
df.fillna(0)

# Create x(predictors) and y (expected outcome)
x,y = to_xy(df, "Label")

float64


In [18]:
poly = PolynomialFeatures(degree=3, interaction_only=True)
x = poly.fit_transform(x)

In [19]:
x.shape

(1000, 2952)

In [ ]:



x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.25, random_state=42)
x_train = preprocessing.StandardScaler().fit_transform(x_train)
x_test = preprocessing.StandardScaler().fit_transform(x_test)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.float32)


#encode_numeric_zscore_all(step1predictions_ytest)
#x_test = np.concatenate((x_test, step1predictions_ytest), axis=1)
#x_test = x_test.astype(np.float32)
#step2predictions_ytest = stack_prediction(x_test, y_test, step2_learners)

